## Описание методики расчета температуры в Hasan_Kabir_class (пока единственной)

Источник:

Al-Safran E. M., Brill J. P. Applied multiphase flow in pipes and flow assurance: oil and gas production. – SPE, 2017.

###  Основное уравнение:



$$T_f = (T_{ei} - g_G L sin\theta) + (T_{fi} - T_{ei})e^{-L/A} + A (1 - e^{-L /A })(g_G sin\theta - \frac{gsin\theta}{C_p} + F_c)$$



### Расчет конвективного теплообмена в скважине

1. Расчет скорости  смеси и содержание жидкости в потоке без учета проскальзывания:

$$ A_p = \frac{\pi}{4}d^2 $$

$$ v_{SL} = \frac{q_L}{A_p} $$

$$ v_{Sg} = \frac{q_g}{A_p} $$

$$ v_m = v_{SL} + v_{Sg} $$

$$ \lambda_L = \frac{v_{SL}}{v_m} $$

2. Расчет физических и термических свойств смеси:

$$ \rho_n = \rho_L \lambda_L + \rho_g (1 - \lambda_L)$$

$$\mu_n = \mu_L \lambda_L + \mu_g (1 - \lambda_L)$$

$$k_n = k_L \lambda_L + k_g (1 - \lambda_L)$$

$$C_{p_n} = \frac {[C_{p_L} \rho_L \lambda_L +C_{p_g} \rho_g (1 - \lambda_L)]} {\rho_n} $$

3. Расчет числа Рейнольдса, Прандтля и Нуссельта

$$N_{Re_n} = \frac{\rho_n v_m d _{ti}}{\mu_n}$$

$$N_{Pr_n} = \frac{\mu_n C _{P_n}}{k_n}$$

$$N_{Nu} = 0.023 N_{Re_n} ^ {0.8} N_{Pr_n} ^ {0.3}$$

$$h_f = \frac{N_{Nu}k_n}{d_{ti}}$$

### Естественная конвекция в затрубном пространстве

1. Расчет безразмерных чисел Прандтля и Грасгофа для затрубного пространства

$$N_{Pr} = \frac{\mu_{an} C _{P_{an}}}{k_{an}}$$

2. Предполагаем неизменяемость физических и термических свойств флюида в затрубном пространстве и принимаем:

$$(T_{to} - T_{ci}) = 3^{\circ}$$

и можем рассчитать:

$$N_{Gr} = \frac {(r_{ci} - r_{to})^3 \bar{\rho}_{an}^2 \beta g (T_{to} - T_{ci}) }{\bar{\mu}_{an}^2}$$

3. Рассчитаем коэффициент естественной конвекции:

$$h_{an} = \frac{0.049(N_{Gr} N_{Pr}) ^ {(1/3)} N_{Gr} ^ {0.074} k_{an}}{r_{to}ln(\frac{r_{ci}}{r_{to}})}$$

4. Скорректируем коэффициент для учета длины трубы

$$h_{an}^{*} = 0.25 h_{an}$$

### Определение общего коэффициента теплопередачи:

1. Рассчитываем безразмерное время и безразмерный коэффциент теплопроводности Земли

$$ t_{Dw} = \frac{k_e t}{\rho_e C_{p_e}r_{w}^2} $$

$$t_{Dw} \leq 1.5  \to f(t) = 1.1281\sqrt {t_{Dw}} ( 1 - 0.3 \sqrt {t_{Dw}} )$$

$$t_{Dw} > 1.5  \to f(t) = [0.4063 + 0.5 ln(t_{Dw})] ( 1 - 0.6 /t_{Dw} )$$

2. Определяем термическое сопротивление элементов

$$(R_{conv})_f = \frac{1}{r_{ti}h_f}$$

$$(R_{cond})_t  = \frac{ln(r_{to}/r_{ti})}{k_t}$$

$$(R_{conv})_{an} = \frac{1}{r_{ci}h_{an}}$$

$$(R_{cond})_c  = \frac{ln(r_{co}/r_{ci})}{k_c}$$

$$(R_{cond})_{cem}  = \frac{ln(r_{w}/r_{co})}{k_{cem}}$$

$$(R_{cond})_{e}  = \frac{f(t)}{k_{e}}$$

3. Определяем общий коэффициент теплопередачи

$$U  = \frac{1}{r_{to}}[\frac{1}{ (R_{conv})_f + (R_{cond})_t + (R_{conv})_{an} +(R_{cond})_c + (R_{cond})_{cem}+(R_{cond})_{e} }]$$


### Итоговый набор формул:

1. Рассчитываем газовый фактор и общий массовый расход смеси:

$$R_p = \frac{q_g}{q_L}$$

$$\dot{m}_T = \rho_n v_m A_p$$

2. Рассчитываем еще параметры:

$$A = \frac{\dot{m}_T  C_{pm}}{\pi d_{to}U}$$

$$F_c = -2.978*10^{-3} + 1.006 * 10 ^ {-6} p_{wh} + 1.906 *10^{-4}\dot{m}_T -1.047 * 10^{-6} R_p+ 3.229 * 10^{-5} \gamma_{API} + 4.009 * 10^{-3} \gamma_g - 0.3551g_G$$

3. Рассчитаем температуру флюида в искомой точке:

$$T_f = (T_{ei} - g_G L sin\theta) + A (1 - e^{-L /A })(g_G sin\theta - \frac{gsin\theta}{C_p} + F_c)$$

4. Определим величину теплового потока

$$T_{wh} = T_r - g_G L sin \theta$$

$$q = 2 \pi r_{to}U (T_f - T_{wh})$$

5. Расчитаем перепад температур в затрубном пространстве

$$(T_{to} - T_{ci})_C = \frac{q}{2\pi r_{ti} h_{an}}$$

6. Повторяем весь цикл расчета пока не будет изменяться разница температур в затрубном пространстве





## Пример использования текущей методики расчета температуры

расчет производится уже проинтегрированным способом, выражение для расчета температуры движущегося флюида предполагает неизменность следующих свойств:
$$ U, C_p, \eta, g_G, \theta, v, dv/dt, dp/dL
$$
на рассматриваемом расчетном участке

Импорт необходимых модулей

In [1]:
import sys
sys.path.append('../')

import uPVT.PVT as PVT
import Hasan_Kabir_class as HK_temp
import Tool_work_with_data as tool

import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import tools

init_notebook_mode(connected=True)

Создание экземляров классов для корреляции и работы с данными

In [2]:
temp_cor = HK_temp.Hasan_Kabir_cor()
temp_data = tool.Data()

fluid = PVT.FluidStanding()

Основной расчет, без шагов

In [3]:

depth_m = 2550
pbh_pa=25*10**6
step_m = 10

pressure_grad_pam=pbh_pa/depth_m

temp_data.clear_data()

distance_list = list(range(1,depth_m,step_m))


for i in distance_list:
    p_pa = pbh_pa - pressure_grad_pam * i
    
    temp_geo_c = temp_cor.tei_c - temp_cor.gg_cm * i
    
    t_fluid_c = temp_cor.calc_t_c_fluid(i, p_pa)
    
    
    temp_data.get_data(temp_cor)
    
    temp_data.distance_list.append(i)
    
    temp_data.t_list.append(temp_geo_c)
    
    temp_data.p_list.append(p_pa / 10 ** 6)


Дополнительные функции для удобства построения графиков

In [4]:
def trace(number_param):
    tracep = go.Scattergl(
        x = temp_data.get_values(number_param),
        y = temp_data.distance_list,
        name = temp_data.get_name(number_param),
        mode = 'markers'
    )
    return tracep

def plot():
    #layout = dict(title = ' H(P,T)',  yaxis=dict(autorange='reversed'))
    layout = dict(title = 'Распредление температуры',  yaxis=dict(title='Расстояние от забоя, м'))
    fig = dict(data=data, layout=layout)

    iplot(fig, filename='basic-scatter')

Все данные, сохраненные с помощью инструмента и доступные для построения графиков.

In [5]:
temp_data.print_all_names()

Номер 0 для получения параметра ql_m3sec
Номер 1 для получения параметра qg_m3sec
Номер 2 для получения параметра kl_wmc
Номер 3 для получения параметра kg_wmc
Номер 4 для получения параметра rhol_kgm3
Номер 5 для получения параметра rhog_kgm3
Номер 6 для получения параметра cpl_jkgc
Номер 7 для получения параметра cpg_jkgc
Номер 8 для получения параметра mul_pas
Номер 9 для получения параметра mug_pas
Номер 10 для получения параметра rti_m
Номер 11 для получения параметра rto_m
Номер 12 для получения параметра rci_m
Номер 13 для получения параметра mu_an_pas
Номер 14 для получения параметра cp_an_jkgc
Номер 15 для получения параметра rho_an_kgm3
Номер 16 для получения параметра k_an_wmc
Номер 17 для получения параметра betta_1c
Номер 18 для получения параметра delta_temp_an_c
Номер 19 для получения параметра rco_m
Номер 20 для получения параметра rwb_m
Номер 21 для получения параметра kcem_wmc
Номер 22 для получения параметра kt_wmc
Номер 23 для получения параметра ke_wmc
Номер 24 для

Построение графиков неправильным способом, без разбивок на участки

In [6]:
trace1 = go.Scattergl(
    x = temp_data.p_list,
    y = temp_data.distance_list,
    name = 'Pressure_MPa',
    mode = 'markers'
)

trace2 = go.Scattergl(
    x = temp_data.t_list,
    y = temp_data.distance_list,
    name = 'T_geo_C',
    mode = 'markers'
)

trace3 = go.Scattergl(
    x = temp_data.get_values(52),
    y = temp_data.distance_list,
    name = temp_data.get_name(52),
    mode = 'markers'
)

data = [trace1, trace2, trace3]

plot()

### В данном расчете применяется та же формула, но расчет ведется отдельным элементам, с пересчетом некоторых PVT свойств

In [7]:
temp_cor_with_step = HK_temp.Hasan_Kabir_cor()
temp_data_with_step = tool.Data()
temp_data_with_step.clear_data()

depth_m = 2550
pbh_pa=25*10**6

pressure_grad_pam=pbh_pa/depth_m
step_m = 10
distance_list = list(range(0, depth_m, step_m))



temp_cor_with_step.tfi_c = 93.3
temp_data_with_step.last_distance_m = 0
for i in distance_list:
    p_pa = pbh_pa - pressure_grad_pam * i

    temp_geo_c = temp_cor.tei_c - temp_cor_with_step.gg_cm * i

    fluid.calc(p_pa / 10 ** 5, temp_cor_with_step.tfi_c)

    temp_cor_with_step.rhol_kgm3 = fluid.rho_oil_kgm3

    temp_cor_with_step.rhog_kgm3 = fluid.rho_gas_kgm3

    temp_cor_with_step.mul_pas = fluid.mu_oil_cP / 10 ** 3
    temp_cor_with_step.gas_pas = fluid.mu_gas_cP / 10 ** 3
    
    distance_in_this_iter = i - temp_data_with_step.last_distance_m

    t_fluid_c = temp_cor_with_step.calc_t_c_fluid(distance_in_this_iter, p_pa)

    temp_data_with_step.last_distance_m = i

    temp_data_with_step.get_data(temp_cor_with_step)

    temp_data_with_step.distance_list.append(i)

    temp_data_with_step.t_list.append(temp_geo_c)

    temp_data_with_step.p_list.append(p_pa / 10 ** 6)

    temp_cor_with_step.tfi_c = t_fluid_c
    
    temp_cor_with_step.tei_c = temp_geo_c



Различные графики и свойства.
Т.к. не все свойства пересчитываются на каждом шаге идет сближение геотермы и температуры флюида ближе к устью

In [8]:
trace1 = go.Scattergl(
    x = temp_data_with_step.p_list,
    y = temp_data_with_step.distance_list,
    name = 'P_MPa',
    mode = 'markers'
)

trace2 = go.Scattergl(
    x = temp_data_with_step.t_list,
    y = temp_data_with_step.distance_list,
    name = 'T_geo_C',
    mode = 'markers'
)

trace3 = go.Scattergl(
    x = temp_data_with_step.get_values(52),
    y = temp_data_with_step.distance_list,
    name = temp_data_with_step.get_name(52) + ' по участкам',
    mode = 'markers'
)

trace4 = go.Scattergl(
    x = temp_data.get_values(52),
    y = temp_data.distance_list,
    name = temp_data.get_name(52) + ' без участков' ,
    mode = 'markers'
)

data = [trace1, trace2, trace3, trace4]

plot()

In [9]:


trace2 = go.Scattergl(
    x = temp_data_with_step.get_values(50),
    y = temp_data_with_step.distance_list,
    name = temp_data_with_step.get_name(50) + ' по участкам',
    mode = 'markers'
)

trace3 = go.Scattergl(
    x = temp_data.get_values(50),
    y = temp_data.distance_list,
    name = temp_data_with_step.get_name(50) + ' без участков',
    mode = 'markers'
)


data = [ trace2, trace3]

plot()
